In [1]:
import pandas as pd


In [2]:


b0 = pd.read_json('openai-batch/response/batch_0.jsonl', lines=True)
b1 = pd.read_json('openai-batch/response/batch_1.jsonl', lines=True)
b2 = pd.read_json('openai-batch/response/batch_2.jsonl', lines=True)

In [3]:
from tqdm.notebook import tqdm
tqdm.pandas()


def extract_embedding(response):
    embed = response['body']['data'][0]['embedding']
    return embed

b0['embeds'] = b0['response'].progress_apply(extract_embedding)
b1['embeds'] = b1['response'].progress_apply(extract_embedding)
b2['embeds'] = b2['response'].progress_apply(extract_embedding)


  0%|          | 0/38124 [00:00<?, ?it/s]

  0%|          | 0/38123 [00:00<?, ?it/s]

  0%|          | 0/38123 [00:00<?, ?it/s]

In [4]:
FIELDS = ['custom_id','embeds']

df_all_em = pd.concat([b0[FIELDS],b1[FIELDS], b2[FIELDS]], axis=0)
df_all_em

,custom_id,embeds
0,0,"[-0.0011454681, -0.010030255, -0.0147224840000..."
1,1,"[-0.028613817000000003, 0.017872352, -0.012471..."
2,2,"[-0.028032498000000003, 0.007056792300000001, ..."
3,3,"[0.00974463, -0.015603273, 0.0013617638, 0.004..."
4,4,"[-0.01259962, -0.005183528000000001, -0.016033..."
...,...,...
38118,114365,"[0.020275634, -0.004595624, -0.004954088000000..."
38119,114366,"[0.006702404, 0.027326606000000003, -0.0025326..."
38120,114367,"[0.022397906000000002, 0.007563821500000001, -..."
38121,114368,"[-0.0052431915, -0.0050309882, -0.0072267037, ..."


In [5]:
df_all_em = df_all_em.reset_index(drop=True)[['embeds']]
df_all_em

,embeds
0,"[-0.0011454681, -0.010030255, -0.0147224840000..."
1,"[-0.028613817000000003, 0.017872352, -0.012471..."
2,"[-0.028032498000000003, 0.007056792300000001, ..."
3,"[0.00974463, -0.015603273, 0.0013617638, 0.004..."
4,"[-0.01259962, -0.005183528000000001, -0.016033..."
...,...
114365,"[0.020275634, -0.004595624, -0.004954088000000..."
114366,"[0.006702404, 0.027326606000000003, -0.0025326..."
114367,"[0.022397906000000002, 0.007563821500000001, -..."
114368,"[-0.0052431915, -0.0050309882, -0.0072267037, ..."


In [6]:
df_ub = pd.read_parquet('data/username_biography_2em.parquet')
df_uc = pd.read_parquet('data/username_caption_2em.parquet')

df_ub = df_ub.reset_index(drop=True)
df_ub.columns = ['username','text']
df_ub = df_ub.dropna()

df_uc = df_uc.reset_index(drop=True)
df_uc.columns = ['username','text']
df_all_no_em = pd.concat([df_ub, df_uc], axis=0).reset_index(drop=True)


df_all_all = pd.concat([df_all_no_em, df_all_em], axis=1)
df_all_all

,username,text,embeds
0,deparmedya,#mediaplanning #mediabuying #sosyalmedya,"[-0.0011454681, -0.010030255, -0.0147224840000..."
1,kafesfirin,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,"[-0.028613817000000003, 0.017872352, -0.012471..."
2,vimerang,Dijital İletişim Yönetimi🎬info@vimerang.comq,"[-0.028032498000000003, 0.007056792300000001, ..."
3,mustafa_yalcinn38,Talas Belediye Başkanı,"[0.00974463, -0.015603273, 0.0013617638, 0.004..."
4,drtubagunebak,👩🏼‍⚕️Ankara ofis:Beytepe\n👩🏼‍💼İstanbul ofis:Ba...,"[-0.01259962, -0.005183528000000001, -0.016033..."
...,...,...,...
114365,tariszeytinyagi,"9 Eylül güzel İzmir’imizin kurtuluş günü, Cumh...","[0.020275634, -0.004595624, -0.004954088000000..."
114366,tariszeytinyagi,Tariş Zeytinyağı sofralarımıza uzanan şifa ve ...,"[0.006702404, 0.027326606000000003, -0.0025326..."
114367,tariszeytinyagi,Kendine özgün tadı ile damağınızda derin bir l...,"[0.022397906000000002, 0.007563821500000001, -..."
114368,tariszeytinyagi,"Serinleten Ayran Çorbası\nMalzemeler:\n1,5 su ...","[-0.0052431915, -0.0050309882, -0.0072267037, ..."


In [7]:
from consts import *

df_ub_em = df_all_all[:3253]
df_ub_em.columns = ['username','biography','embeds']
df_ub_em = df_ub_em.reset_index(drop=True)
df_ub_em.to_parquet(get_filename(TT.BIO, EMBED_PROVIDER.OPENAI))

In [8]:
df_uc_em = df_all_all[3253:]
df_uc_em.columns = ['username','caption','embeds']
df_uc_em = df_uc_em.reset_index(drop=True)
df_uc_em.to_parquet(get_filename(TT.CAP, EMBED_PROVIDER.OPENAI))